In [1]:
import pickle
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import chi2
import numpy as np

In [2]:

data = pd.read_csv('C:\\Users\\baril\\OneDrive\\Desktop\\3395Compe\\train.csv', sep = ',', header=0).to_numpy()
data = pd.DataFrame(data = data[:,1:3], columns = ["Article", "Categorie"])
#data.head()


In [3]:

data['Article_mod'] = data['Article'].str.replace("\r", " ")
data['Article_mod'] = data['Article_mod'].str.replace("\n", " ")
data['Article_mod'] = data['Article_mod'].str.replace("    ", " ")
data['Article_mod'] = data['Article_mod'].str.replace('"', '')
data['Article_mod'] = data['Article_mod'].str.lower()
punctuation_signs = list("?:!.,;")

for punct_sign in punctuation_signs:
    data['Article_mod'] = data['Article_mod'].str.replace(punct_sign, '')
data['Article_mod'] = data['Article_mod'].str.replace("'s", "")


In [4]:
nltk.download('punkt')
nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()
n_obs = len(data)
word_bag_list = [] #tous les sacs du DF

for ligne in range(n_obs):
    word_bag = [] #sac de mot par article
    
    texte= data.loc[ligne]['Article_mod']
    mots = texte.split(" ")

    for mot in mots:
        word_bag.append(lemmatizer.lemmatize(mot, pos="v"))
        
    text_split = " ".join(word_bag)
    
    word_bag_list.append(text_split)

data['Article_mod'] = word_bag_list

[nltk_data] Downloading package punkt to C:\Users\baril/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\baril/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [5]:
nltk.download('stopwords')

no_word = list(stopwords.words('english'))

for mot in no_word:
    reg = r"\b" + mot + r"\b"
    data['Article_mod'] = data['Article_mod'].str.replace(reg, '')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\baril/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
list_columns = ["Categorie", "Article", "Article_mod"]
data = data[list_columns]
np.unique(data.Categorie)
categorie_codes = {
    'astro-ph': 0,
    'astro-ph.CO': 1,
    'astro-ph.GA': 2,
    'astro-ph.SR': 3,
    'cond-mat.mes-hall': 4,
    'cond-mat.mtrl-sci': 5,
    'cs.LG': 6,
    'gr-qc': 7,
    'hep-ph' : 8,
    'hep-th' : 9,
    'math.AP' : 10,
    'math.CO' : 11,
    'physics.optics' : 12,
    'quant-ph' : 13,
    'stat.ML' : 14
}
data['Categorie_Codes'] = data['Categorie']
data = data.replace({'Categorie_Codes':categorie_codes})
#data.head()

In [15]:
train_x, test_x, train_y, test_y = train_test_split(data['Article_mod'], 
                                                   data['Categorie_Codes'],
                                                   test_size=0.2,
                                                   random_state=11)  

test_x_ast = test_x
test_y_ast = test_y
train_x_ast = train_x
train_y_ast = train_y

#TFIDF
ngram_range = (1,2)
min_df = 10
max_df = 1.
max_features = 3000

tfidf = TfidfVectorizer(encoding='utf-8',
                        ngram_range=(1,2),
                        stop_words=None,
                        lowercase=False,
                        max_df=max_df,
                        min_df=min_df,
                        max_features=max_features, ##à jouer avec
                        norm='l2',
                        sublinear_tf=True)
                        
train_x = tfidf.fit_transform(train_x).toarray()
test_x = tfidf.transform(test_x).toarray()

In [16]:
from sklearn import svm
from sklearn.metrics import accuracy_score
np.random.seed(42)

SVM = svm.SVC(kernel='rbf')
SVM.fit(train_x,train_y)

predictions_SVM = SVM.predict(test_x)
   
print("SVM Accuracy Score (Validation) -> ",accuracy_score(predictions_SVM, test_y)*100)
predictions_SVM2 = SVM.predict(train_x)
print("SVM Accuracy Score (Train) -> ",accuracy_score(predictions_SVM2, train_y)*100)



SVM Accuracy Score (Validation) ->  80.86666666666666
SVM Accuracy Score (Train) ->  98.83333333333333


In [17]:
#TFIDF
ngram_range = (1,1)
min_df = 1
max_df = 125
max_features = 10000

tfidf = TfidfVectorizer(encoding='utf-8',
                        ngram_range=ngram_range,
                        stop_words=None,
                        lowercase=False,
                        max_df=max_df,
                        min_df=min_df,
                        max_features=max_features, ##à jouer avec
                        norm='l2',
                        sublinear_tf=True)
train_x_ast = tfidf.fit_transform(train_x_ast).toarray()
test_x = tfidf.transform(test_x_ast).toarray()


In [ ]:
SVMAstro = svm.SVC(kernel='rbf')
SVMAstro.fit(train_x_ast, train_y_ast)
test_x_ast = tfidf.transform(test_x_ast).toarray()
pred = SVMAstro.predict(test_x_ast)
print("SVM Accuracy Score (Validation) -> ",accuracy_score(pred, test_y_ast)*100)


In [ ]:
len(predictions_SVM)


In [ ]:
len(pred)

In [ ]:
for i in range(len(predictions_SVM)):
    if predictions_SVM[i] <3:
               predictions_SVM[i] = pred[i]

print("SVM Accuracy Score (Validation) -> ",accuracy_score(predictions_SVM, test_y)*100)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
aux_df = data[['Categorie', 'Categorie_Codes']].drop_duplicates().sort_values('Categorie_Codes')
conf_matrix = confusion_matrix(test_y, predictions_SVM)
plt.figure(figsize=(12.8,6))
sns.heatmap(conf_matrix, 
            annot=True,
            xticklabels=aux_df['Categorie'].values, 
            yticklabels=aux_df['Categorie'].values,
            cmap="Blues")
plt.ylabel('Predicted')
plt.xlabel('Actual')
plt.title('Confusion matrix')
plt.show()